<a href="https://colab.research.google.com/github/TheNeodev/RVC-MAKER/blob/main/webui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This project was created by [TheNeoDev](https://github.com/TheNeodev)** base on [PhamHuynhAnh16/Vietnamese-RVC](https://github.com/PhamHuynhAnh16/Vietnamese-RVC)

---

<big>**Please do not use the project for any purposes that violate ethics, laws, or cause harm to individuals, organizations...**

---
 **In case the user does not comply with the terms or violates them, I will not be responsible for any claims, damages, or legal liabilities, whether in contract, due to negligence, or other reasons, arising from, out of, or related to the software, the use of the software, or other transactions related to the software.**</big>

In [ ]:
#@title **🌟Settings**

import os
from ipywidgets import Button
from IPython.display import clear_output
print("👩🏻‍💻 Settings...")

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

!git clone https://github.com/TheNeodev/RVC-MAKER.git /content/RVC-MAKER > /dev/null 2>&1
!pip install -r /content/Vietnamese_RVC/requirements.txt --no-cache-dir -q > /dev/null 2>&1

#@markdown **💻 Installation will take about 2 minutes to complete!**
clear_output()
Button(description="\u2714 done!", button_style="success")

In [ ]:
#@title **Run webui**
import os
import shutil

%cd /content/Vietnamese_RVC
#@markdown **To experience all the features, use the interface :) If you want simplicity, don't use the interface**

#@markdown **If you know you can use the chart to check for overtraining 👍**
tensorboard = False #@param {type:"boolean"}

if tensorboard:
  %load_ext tensorboard
  %tensorboard --logdir ./assets/logs --port=6870

!python main/app/app.py --share